# Working with 13-F Data

In [27]:
from bs4 import BeautifulSoup

f = open("Data/13F/File1.html", "r")
soup = BeautifulSoup(f, 'lxml')

NameError: name 'displot' is not defined

In [10]:
#One of the first things we can find is the company name
print(soup.find("span", {'class': 'companyName'}).text)
print()
#Let's truncate up to (Filer)
company_name = soup.find("span", {'class': 'companyName'}).text
print(company_name[:company_name.index(" (Filer)")].strip())

HIGHFIELDS CAPITAL MANAGEMENT LP (Filer)
 CIK: 0001079563 (see all company filings)

HIGHFIELDS CAPITAL MANAGEMENT LP


In [11]:
#The divs with class mailer hold the addresses
addresses = soup.find_all("div", {'class': 'mailer'})
print(addresses)

[<div class="mailer">Mailing Address
      <span class="mailerAddress">200 CLARENDON STREET 59TH FLOOR</span>
<span class="mailerAddress">
BOSTON MA 02116      </span>
</div>, <div class="mailer">Business Address
      <span class="mailerAddress">200 CLARENDON STREET 59TH FLOOR</span>
<span class="mailerAddress">
BOSTON MA 02116      </span>
<span class="mailerAddress">6178507500</span>
</div>]


In [12]:
#The first is the mailing address and we can split it into the text of the different lines in span
mailing_address = addresses[0]
mailing_address = [x.text.strip() for x in mailing_address.find_all('span')]
print(mailing_address)

['200 CLARENDON STREET 59TH FLOOR', 'BOSTON MA 02116']


In [13]:
#Same for business address, the last line is the telephone number
business_address = addresses[1]
business_address = [x.text.strip() for x in business_address.find_all('span')]
print(business_address)

['200 CLARENDON STREET 59TH FLOOR', 'BOSTON MA 02116', '6178507500']


In [14]:
#We might not have only US companies, so we want to make sure we use a flexible way 
#to get the zipcode and state out of this
#Regular expressions will give us that

import re

#Regular expression to find two letters followed by a space and 5 numbers
pattern = re.compile('[A-Z]{2}\s[0-9]{5}')

#Search the string
zip_state = re.search(re.compile('[A-Z]{2}\s[0-9]{5}'), business_address[-2])
print(zip_state)

if zip_state is not None:
    state = business_address[-2][zip_state.span()[0]:zip_state.span()[1]][:2]
    print(state)
    
    zip_code = business_address[-2][zip_state.span()[0]:zip_state.span()[1]][3:]
    print(zip_code)

<re.Match object; span=(7, 15), match='MA 02116'>
MA
02116


In [15]:
#We also want to scrape the holdings document, first find the url
table_file = soup.find("table", {"class": 'tableFile'})
filing_url_13f = 'https://www.sec.gov' + table_file.find_all('tr')[3].find('a')['href']
print(filing_url_13f)

https://www.sec.gov/Archives/edgar/data/1079563/000106299319002337/xslForm13F_X01/form13fInfoTable.xml


In [16]:
import requests
soup2 = BeautifulSoup(requests.get(filing_url_13f).content, 'lxml')
print(soup2)

<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">
<html xmlns="http://www.w3.org/1999/xhtml">
<head>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<title>SEC.gov | Request Rate Threshold Exceeded</title>
<style>
html {height: 100%}
body {height: 100%; margin:0; padding:0;}
#header {background-color:#003968; color:#fff; padding:15px 20px 10px 20px;font-family:Arial, Helvetica, sans-serif; font-size:20px; border-bottom:solid 5px #000;}
#footer {background-color:#003968; color:#fff; padding:15px 20px;font-family:Arial, Helvetica, sans-serif; font-size:20px;}
#content {max-width:650px;margin:60px auto; padding:0 20px 100px 20px; background-image:url(seal_bw.png);background-repeat:no-repeat;background-position:50% 100%;}
h1 {font-family:Georgia, Times, serif; font-size:20px;}
h2 {text-align:center; font-family:Georgia, Times, serif; font-size:20px; width:100%; border-bottom:solid #999 1px;padding-

In [17]:
#The holdings can be found from one of the table tags
holdings = soup2.find('table', {'summary':'Form 13F-NT Header Information'})
print(holdings)

None


In [18]:
#Let's pull the table into a pandas dataframe, but skip the first three rows
import pandas as pd
holdings = pd.DataFrame([list(x.find_all('td')) for x in holdings.find_all('tr')[3:]])
print(holdings)

AttributeError: 'NoneType' object has no attribute 'find_all'

In [19]:
#The apply_map function can be used to apply a function to every cell
holdings = holdings.applymap(lambda x: x.text)
print(holdings)

AttributeError: 'NoneType' object has no attribute 'applymap'

In [20]:
import numpy as np
#And do some clean up while we are at it
holdings.columns = ["Issuer", "Title", "CUSIP", "Market Value",
                           "Amount", "Type Security", "Put/Call", "Investment Discretion",
                           "Manager", "Sole", "Shared", "Other"]
#Wherever there is blank space convert it to null for the Put/Call field
holdings["Put/Call"] = holdings["Put/Call"].replace('\xa0', np.NaN)

#Convert market value to a float by removing the commas and casting as a float
holdings["Market Value"] = holdings["Market Value"].str.replace(",","").astype(float)

print(holdings)

AttributeError: 'NoneType' object has no attribute 'columns'

In [21]:
#We can also have a sub table, stock holdings where we get rid of any options
stock_holdings = holdings[pd.isnull(holdings["Put/Call"])]
print(stock_holdings)

TypeError: 'NoneType' object is not subscriptable

In [22]:
#Put it all together into a class
class company_15F:
    def __init__(self, filing_page):
        #Initiliaze with the filing page
        self.filing_page = filing_page
        self.extract_company_info()
        self.extract_13f()
        
    def extract_company_info(self):
        addresses = self.filing_page.find_all("div", {'class': 'mailer'})
        
        mailing_address = addresses[0]
        self.mailing_address = [x.text.strip() for x in mailing_address.find_all('span')]
        
        business_address = addresses[1]
        self.business_address = [x.text.strip() for x in business_address.find_all('span')]
        
        #Regular expression to find two letters followed by a space and 5 numbers
        pattern = re.compile('[A-Z]{2}\s[0-9]{5}')

        #Search the string
        zip_state = re.search(re.compile('[A-Z]{2}\s[0-9]{5}'), self.business_address[-2])

        if zip_state is not None:
            self.state = self.business_address[-2][zip_state.span()[0]:zip_state.span()[1]][:2]

            self.zip_code = self.business_address[-2][zip_state.span()[0]:zip_state.span()[1]][3:]
        else:
            self.state = None
            self.zip_code = None
        company_name = self.filing_page.find("span", {'class': 'companyName'}).text
        self.company_name = company_name[:company_name.index(" (Filer)")]
        
        table_file = self.filing_page.find("table", {"class": 'tableFile'})
        self.filing_url_13f = 'https://www.sec.gov' + table_file.find_all('tr')[3].find('a')['href']
        
    def extract_13f(self):
        soup = BeautifulSoup(requests.get(self.filing_url_13f).content, 'lxml')
        holdings = soup.find('table', {'summary':'Form 13F-NT Header Information'})
        holdings = pd.DataFrame([list(x.find_all('td')) for x in holdings.find_all('tr')[3:]]).applymap(lambda x: x.text)
        holdings.columns = ["Issuer", "Title", "CUSIP", "Market Value",
                           "Amount", "Type Security", "Put/Call", "Investment Discretion",
                           "Manager", "Sole", "Shared", "Other"]
        holdings["Put/Call"] = holdings["Put/Call"].replace('\xa0', np.NaN)
        holdings["Market Value"] = holdings["Market Value"].str.replace(",","").astype(float)
        self.holdings = holdings
        self.stock_holdings = self.holdings[pd.isnull(self.holdings["Put/Call"])]

        
f = open("Data/13F/File1.html", "r")
soup = BeautifulSoup(f, 'lxml')
company = company_15F(soup)

AttributeError: 'NoneType' object has no attribute 'find_all'

In [23]:
print(company.company_name)
print(company.state)
print(company.stock_holdings)

NameError: name 'company' is not defined

In [24]:
#Let's iterate through the files and put them into a list
import os
data = []
for file_name in os.listdir("Data/13F/"):
    #Use a try statement because some may fail and we don't have time to fix all errors
    try:
        f = open("Data/13F/{}".format(file_name), "r")
        soup = BeautifulSoup(f, 'lxml')
        company = company_15F(soup)
        data.append(company)
    except:
        print(file_name)
        pass

File0.html
File1.html
File10.html
File11.html
File12.html
File13.html
File14.html
File15.html
File16.html
File17.html
File18.html
File19.html
File2.html
File20.html
File21.html
File22.html
File23.html
File24.html
File25.html
File26.html
File27.html
File28.html
File29.html
File3.html
File30.html
File31.html
File32.html
File33.html
File34.html
File35.html
File36.html
File37.html
File38.html
File39.html
File4.html
File40.html
File41.html
File42.html
File43.html
File44.html
File45.html
File46.html
File47.html
File48.html
File49.html
File5.html
File50.html
File51.html
File52.html
File53.html
File54.html
File55.html
File56.html
File57.html
File58.html
File59.html
File6.html
File60.html
File61.html
File62.html
File63.html
File64.html
File65.html
File66.html
File67.html
File68.html
File69.html
File7.html
File70.html
File71.html
File72.html
File73.html
File74.html
File75.html
File76.html
File77.html
File78.html
File79.html
File8.html
File80.html
File9.html


# Comparing Holding Similarities

In [25]:
#Something we can do is get the states and fund values
#Rememeber the following will get the elements from the company
print(data[0].stock_holdings["Market Value"].sum())
print(data[0].state)

IndexError: list index out of range

In [26]:
#Create a dataframe object of it
state_fund_values = pd.DataFrame([[company.stock_holdings["Market Value"].sum(),
                                  company.state] for company in data])
state_fund_values.columns = ["Fund Value", "State"]
print(state_fund_values)

ValueError: Length mismatch: Expected axis has 0 elements, new values have 2 elements

In [50]:
#Get rid of unknown states
state_fund_values = state_fund_values[~pd.isnull(state_fund_values["State"])]

#Get rid of any fund values of 0
state_fund_values = state_fund_values[state_fund_values["Fund Value"] != 0]

#Groupby state then reset index
state_fund_values = state_fund_values.groupby("State").sum().reset_index()

#Add in a column for the log of fund value
state_fund_values["Log Fund Value"] = np.log(state_fund_values["Fund Value"])
print(state_fund_values)

   State   Fund Value  Log Fund Value
0     AL   11600486.0       16.266558
1     CA  812797086.0       20.515992
2     CT   93571942.0       18.354241
3     DC    1631726.0       14.305149
4     GA    1361774.0       14.124299
5     IA     626662.0       13.348163
6     IL      19158.0        9.860476
7     KS    1530468.0       14.241084
8     MA    9084205.0       16.022048
9     MI     207050.0       12.240716
10    MN     387000.0       12.866180
11    MO    1942862.0       14.479673
12    NC    2321093.0       14.657549
13    NE    1181523.0       13.982315
14    NJ   15129456.0       16.532154
15    NY   22345087.0       16.922117
16    OH     371929.0       12.826458
17    OR     432235.0       12.976725
18    RI     222745.0       12.313783
19    TN    1442377.0       14.181803
20    TX    1422787.0       14.168128
21    VA     886709.0       13.695272


In [51]:
#Folium is a library which can let us map things
#For example, a base map
import folium as folium
state_map = folium.Map(location=[38, -100], zoom_start=4)
state_map

In [52]:
#A choropleth will give us a highlighted map, which we can use to see which states have the most fund value
state_map = folium.Map(location=[38, -100], zoom_start=4)

url = 'https://raw.githubusercontent.com/python-visualization/folium/master/examples/data'
state_geo = f'{url}/us-states.json'

folium.Choropleth(
    geo_data=state_geo,
    name='choropleth',
    data=state_fund_values,
    columns=['State', 'Fund Value'],
    key_on='feature.id',
    fill_color='YlGn',
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='Fund Value'
).add_to(state_map)
state_map

In [53]:
#The log of fund value might give us a better idea
state_map = folium.Map(location=[38, -100], zoom_start=4)

url = 'https://raw.githubusercontent.com/python-visualization/folium/master/examples/data'
state_geo = f'{url}/us-states.json'

folium.Choropleth(
    geo_data=state_geo,
    name='choropleth',
    data=state_fund_values,
    columns=['State', 'Log Fund Value'],
    key_on='feature.id',
    fill_color='YlGn',
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='Log Fund Value'
).add_to(state_map)
state_map

In [54]:
#We can also create a master dataframe of all holdings
all_holdings = []
for company in data:
    holdings = company.stock_holdings.copy()
    holdings["Fund"] = company.company_name
    all_holdings.append(holdings)
all_holdings = pd.concat(all_holdings)

#Deal with funds with the same name
all_holdings = all_holdings.groupby(["Issuer", "Fund"]).sum()["Market Value"].unstack().fillna(0)
all_holdings

Fund,"ACR Alpine Capital Research, LLC","AIP, LLC",ALLEN OPERATIONS LLC,AMERITAS INVESTMENT CORP,ARCH VENTURE CORP,"Aquamarine Capital Management, LLC","B. Riley Wealth Management, Inc.","BTC Capital Management, Inc.","BlueMountain Capital Management, LLC",Brahman Capital Corp.,...,"Strategic Point Investment Advisors, LLC","Strategic Value Partners, LLC",Thomasville National Bank,UNITED BANK,V Wealth Advisors LLC,"Varde Management, L.P.","Vulcan Value Partners, LLC","WealthPLAN Partners, LLC",Weiss Asset Management LP,Zhang Financial LLC
Issuer,,,,,,,,,,,,,,,,,,,,,
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1-800 FLOWERS.COM INC CLASS A,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1ST SOURCE CORP,0.0,0.0,0.0,77.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1st Source Corp,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
22ND CENTY GROUP INC,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2U INC,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2U INC COM USD0.001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3 M Company,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3-D SYS CORP DEL,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [55]:
#Let's find the top stocks represented in funds
print((all_holdings > 0).sum(axis=1).sort_values(ascending=False))

Issuer
MICROSOFT CORP                                  21
JOHNSON & JOHNSON                               20
APPLE INC                                       20
PFIZER INC                                      20
PEPSICO INC                                     18
INTEL CORP                                      17
EXXON MOBIL CORP                                17
ABBVIE INC                                      17
AT&T INC                                        17
AMAZON COM INC                                  16
HONEYWELL INTL INC                              15
MERCK & CO INC                                  15
VERIZON COMMUNICATIONS INC                      15
ALPHABET INC                                    15
AMGEN INC                                       15
DOWDUPONT INC                                   15
NVIDIA CORP                                     15
UNITEDHEALTH GROUP INC                          14
CONOCOPHILLIPS                                  14
VISA INC                

In [56]:
#Something interesting we could do is visualize where the top 5 stocks are most represented for the states

#Pull the top 5
top_5_stocks = (all_holdings > 0).sum(axis=1).sort_values(ascending=False).head(5).index
print(top_5_stocks)

Index(['MICROSOFT CORP', 'JOHNSON & JOHNSON', 'APPLE INC', 'PFIZER INC',
       'PEPSICO INC'],
      dtype='object', name='Issuer')


In [57]:
#Index the holdings to be only the top 5
top_5_holdings = all_holdings.loc[top_5_stocks].stack().reset_index()
top_5_holdings.columns = ["Stock", "Fund", "Holding"]
print(top_5_holdings)

              Stock                                            Fund   Holding
0    MICROSOFT CORP                ACR Alpine Capital Research, LLC  120317.0
1    MICROSOFT CORP                                        AIP, LLC       0.0
2    MICROSOFT CORP                            ALLEN OPERATIONS LLC     226.0
3    MICROSOFT CORP                        AMERITAS INVESTMENT CORP    3192.0
4    MICROSOFT CORP                               ARCH VENTURE CORP       0.0
5    MICROSOFT CORP              Aquamarine Capital Management, LLC       0.0
6    MICROSOFT CORP                B. Riley Wealth Management, Inc.    4393.0
7    MICROSOFT CORP                    BTC Capital Management, Inc.   14568.0
8    MICROSOFT CORP            BlueMountain Capital Management, LLC      80.0
9    MICROSOFT CORP                           Brahman Capital Corp.       0.0
10   MICROSOFT CORP                     CANYON CAPITAL ADVISORS LLC       0.0
11   MICROSOFT CORP                            CLEARARC CAPITAL 

In [58]:
#Now let's create a map of fund name to state, we will use first to avoid issues of repeated fund names
fund_to_state_map = pd.DataFrame([[company.company_name, company.state] for company in data])
fund_to_state_map.columns = ["Fund Name", "State"]
fund_to_state_map = fund_to_state_map.groupby("Fund Name").first()
print(fund_to_state_map)

                                               State
Fund Name                                           
ACR Alpine Capital Research, LLC                  MO
AIP, LLC                                          NY
ALLEN OPERATIONS LLC                              NY
AMERITAS INVESTMENT CORP                          NE
ARCH VENTURE CORP                                 IL
Aquamarine Capital Management, LLC                MA
B. Riley Wealth Management, Inc.                  TN
BTC Capital Management, Inc.                      IA
BlueMountain Capital Management, LLC              NY
Brahman Capital Corp.                             NY
CANYON CAPITAL ADVISORS LLC                       CA
CLEARARC CAPITAL INC                              OH
CWM Advisors, LLC                                 CA
Cypress Wealth Advisors LLC                       CA
DIXON HUBARD FEINOUR & BROWN INC/VA               VA
DeepCurrents Investment Group LLC                 NY
EMS Capital LP                                

In [59]:
#Join in the states and drop where we don't have a state
top_5_holdings = top_5_holdings.join(fund_to_state_map, on="Fund").dropna()
print(top_5_holdings)

              Stock                                       Fund   Holding State
0    MICROSOFT CORP           ACR Alpine Capital Research, LLC  120317.0    MO
1    MICROSOFT CORP                                   AIP, LLC       0.0    NY
2    MICROSOFT CORP                       ALLEN OPERATIONS LLC     226.0    NY
3    MICROSOFT CORP                   AMERITAS INVESTMENT CORP    3192.0    NE
4    MICROSOFT CORP                          ARCH VENTURE CORP       0.0    IL
5    MICROSOFT CORP         Aquamarine Capital Management, LLC       0.0    MA
6    MICROSOFT CORP           B. Riley Wealth Management, Inc.    4393.0    TN
7    MICROSOFT CORP               BTC Capital Management, Inc.   14568.0    IA
8    MICROSOFT CORP       BlueMountain Capital Management, LLC      80.0    NY
9    MICROSOFT CORP                      Brahman Capital Corp.       0.0    NY
10   MICROSOFT CORP                CANYON CAPITAL ADVISORS LLC       0.0    CA
11   MICROSOFT CORP                       CLEARARC C

In [60]:
#Finally group by state and stock to find the values in each
top_5_holdings = top_5_holdings.groupby(["Stock", "State"]).sum()[["Holding"]].reset_index()
print(top_5_holdings)

                 Stock State   Holding
0            APPLE INC    AL       0.0
1            APPLE INC    CA   22477.0
2            APPLE INC    CT       0.0
3            APPLE INC    DC       0.0
4            APPLE INC    GA    1979.0
5            APPLE INC    IA   14259.0
6            APPLE INC    IL       0.0
7            APPLE INC    KS    9801.0
8            APPLE INC    MA  168748.0
9            APPLE INC    MI    2086.0
10           APPLE INC    MN       0.0
11           APPLE INC    MO       0.0
12           APPLE INC    NC       0.0
13           APPLE INC    NE   41077.0
14           APPLE INC    NJ  241945.0
15           APPLE INC    NY  183429.0
16           APPLE INC    OH   14256.0
17           APPLE INC    OR   12645.0
18           APPLE INC    RI       0.0
19           APPLE INC    TN   12746.0
20           APPLE INC    TX    3638.0
21           APPLE INC    VA   16468.0
22   JOHNSON & JOHNSON    AL       0.0
23   JOHNSON & JOHNSON    CA    1456.0
24   JOHNSON & JOHNSON   

In [65]:
from IPython.display import display
for x in top_5_holdings["Stock"].unique():
    state_map = folium.Map(location=[38, -100], zoom_start=4)

    url = 'https://raw.githubusercontent.com/python-visualization/folium/master/examples/data'
    state_geo = f'{url}/us-states.json'

    folium.Choropleth(
        geo_data=state_geo,
        name='choropleth',
        data=top_5_holdings[top_5_holdings["Stock"]==x],
        columns=['State', 'Holding'],
        key_on='feature.id',
        fill_color='YlGn',
        fill_opacity=0.7,
        line_opacity=0.2,
        legend_name=x
    ).add_to(state_map)
    display(state_map)

In [32]:
#Define a binary representation of stock owernship
holdings_binary = (all_holdings > 0 ).astype(int)

In [36]:
#One thing we can check is correlation among the fund holdings
holdings_binary.corr()

Fund,"ACR Alpine Capital Research, LLC","AIP, LLC",ALLEN OPERATIONS LLC,AMERITAS INVESTMENT CORP,ARCH VENTURE CORP,"Aquamarine Capital Management, LLC","B. Riley Wealth Management, Inc.","BTC Capital Management, Inc.","BlueMountain Capital Management, LLC",Brahman Capital Corp.,...,"Strategic Point Investment Advisors, LLC","Strategic Value Partners, LLC",Thomasville National Bank,UNITED BANK,V Wealth Advisors LLC,"Varde Management, L.P.","Vulcan Value Partners, LLC","WealthPLAN Partners, LLC",Weiss Asset Management LP,Zhang Financial LLC
Fund,,,,,,,,,,,,,,,,,,,,,
"ACR Alpine Capital Research, LLC",1.000000,-0.000737,0.065560,0.095885,-0.000737,0.054619,0.054703,0.154152,0.068175,-0.002555,...,-0.003042,-0.002085,0.043439,0.064659,0.121584,-0.001277,-0.005435,0.119100,0.054448,0.038872
"AIP, LLC",-0.000737,1.000000,-0.000688,-0.003602,-0.000132,-0.000418,-0.002858,-0.001634,-0.004703,-0.000458,...,-0.000545,-0.000374,-0.001819,-0.001336,-0.001849,-0.000229,-0.000974,-0.001883,-0.002214,-0.001070
ALLEN OPERATIONS LLC,0.065560,-0.000688,1.000000,0.074432,-0.000688,0.058832,0.051344,0.102794,0.057694,0.053315,...,-0.002838,-0.001946,0.033585,0.031670,0.103411,-0.001191,-0.005071,0.087385,0.036181,0.042466
AMERITAS INVESTMENT CORP,0.095885,-0.003602,0.074432,1.000000,-0.003602,0.065072,0.082148,0.324144,0.191333,0.022422,...,-0.014866,-0.010192,0.076321,0.120912,0.368734,-0.006239,-0.026560,0.401120,0.136457,0.046097
ARCH VENTURE CORP,-0.000737,-0.000132,-0.000688,-0.003602,1.000000,-0.000418,-0.002858,-0.001634,-0.004703,-0.000458,...,-0.000545,-0.000374,-0.001819,-0.001336,-0.001849,-0.000229,-0.000974,-0.001883,-0.002214,-0.001070
"Aquamarine Capital Management, LLC",0.054619,-0.000418,0.058832,0.065072,-0.000418,1.000000,0.037538,0.047023,0.016243,-0.001449,...,-0.001725,-0.001183,-0.005756,0.027470,0.063722,-0.000724,-0.003082,0.085193,0.032144,-0.003384
"B. Riley Wealth Management, Inc.",0.054703,-0.002858,0.051344,0.082148,-0.002858,0.037538,1.000000,0.114374,-0.029375,-0.009909,...,-0.011798,-0.008089,0.066565,0.079223,0.172167,-0.004952,-0.021079,0.157134,0.049221,0.025758
"BTC Capital Management, Inc.",0.154152,-0.001634,0.102794,0.324144,-0.001634,0.047023,0.114374,1.000000,0.150028,-0.005665,...,-0.006745,-0.004624,0.124831,0.190004,0.369994,-0.002831,-0.012050,0.380561,0.146071,0.120324
"BlueMountain Capital Management, LLC",0.068175,-0.004703,0.057694,0.191333,-0.004703,0.016243,-0.029375,0.150028,1.000000,0.040530,...,-0.019413,-0.013309,0.006012,0.064087,0.157637,0.010786,-0.034684,0.124194,0.075306,0.002762


In [38]:
#Let's get rid of duplicates by taking only the rows above the diagonal and also get rid of the diagonals

#First grab the indices present in the matrix
rows, cols = np.indices((len(holdings_binary.columns), len(holdings_binary.columns)))

#Diagonal elements indices
print(rows == cols)

#Beneath the matrix indices
print()
print(rows > cols)

#Create the mask, either being on the diagonal or below it
mask = (rows == cols) | (rows > cols)

#Mask the matrix
print(holdings_binary.corr().mask(mask).head(10))
print()
print()
print()

[[ True False False ..., False False False]
 [False  True False ..., False False False]
 [False False  True ..., False False False]
 ..., 
 [False False False ...,  True False False]
 [False False False ..., False  True False]
 [False False False ..., False False  True]]

[[False False False ..., False False False]
 [ True False False ..., False False False]
 [ True  True False ..., False False False]
 ..., 
 [ True  True  True ..., False False False]
 [ True  True  True ...,  True False False]
 [ True  True  True ...,  True  True False]]
Fund                                  ACR Alpine Capital Research, LLC  \
Fund                                                                     
ACR Alpine Capital Research, LLC                                   NaN   
AIP, LLC                                                           NaN   
ALLEN OPERATIONS LLC                                               NaN   
AMERITAS INVESTMENT CORP                                           NaN   
ARCH VENTUR

In [39]:
#Now stack and see the correlations ranked
holdings_binary_correlation = holdings_binary.corr().mask(mask).stack().sort_values(ascending=False)
print(holdings_binary_correlation.head(10))

Fund                                   Fund                                 
METROPOLITAN LIFE INSURANCE CO/NY      MetLife Investment Advisors, LLC         0.985107
M Holdings Securities, Inc.            Pacific Center for Financial Services    0.488645
Pacific Center for Financial Services  V Wealth Advisors LLC                    0.451644
M Holdings Securities, Inc.            V Wealth Advisors LLC                    0.450072
V Wealth Advisors LLC                  WealthPLAN Partners, LLC                 0.448640
M Holdings Securities, Inc.            WealthPLAN Partners, LLC                 0.445804
                                       QUEST CAPITAL MANAGEMENT INC /ADV        0.445731
AMERITAS INVESTMENT CORP               M Holdings Securities, Inc.              0.420274
                                       Pacific Center for Financial Services    0.418864
Pacific Center for Financial Services  WealthPLAN Partners, LLC                 0.414407
dtype: float64
